# Math work

In [52]:
#from PIL import Image
#import sys
#sys.path.append('/usr/local/lib/python3.7/site-packages')
#import os
import numpy as np
import importlib
from numpy import linalg
import PO_helperfuns
from PO_helperfuns import *
import PO_DatFrameFuns
from PO_DatFrameFuns import *
import scipy.stats as scs
import matplotlib as mpl
import matplotlib.pyplot as plt
#importlib.reload(helperfuns)
import pandas as pd
import sympy as sp
from sympy import *
from sympy.abc import lamda, beta
from sympy.solvers import solve
#np.set_printoptions(precision=3, suppress = True)
#import seaborn as sns
# next two libraries are used to flatten list of lists
import functools
import operator
# for formatting tick labels
#from matplotlib.ticker import FormatStrFormatter

#for parallelizing:
import multiprocessing as mp
from pathos.multiprocessing import ProcessingPool as Pool





In [53]:
u1, u2,r1,r2, K, pc,  W = sp.symbols('\hat{u}_1,\hat{u}_2, \hat{r}_1,\hat{r}_2,  K, \pi_C, \hat{W}')
from sympy.abc import beta, lamda



In [54]:
psi1 = u1/(u1 + u2)
psi2 = u2/(u1+u2)
R1 = r1/(r1+r2)
R2 = r2/(r1+r2)
L1 = K*psi1 + pc*R1
L2 = K*psi2 + pc*R2
What = 1 + (r1)*L1 + (r2)*L2
Wu1 = L1*(2-beta*u1)

In [55]:
expr = What*u1 - Wu1
#expr = expr.subs([(r1,1-beta*u1),(r2,1-beta*u2)])

In [56]:
n,d = fraction(simplify(expr)) # n = 0 is one of the equations we solve

In [57]:
expr

\hat{u}_1*(\hat{r}_1*(K*\hat{u}_1/(\hat{u}_1 + \hat{u}_2) + \hat{r}_1*\pi_C/(\hat{r}_1 + \hat{r}_2)) + \hat{r}_2*(K*\hat{u}_2/(\hat{u}_1 + \hat{u}_2) + \hat{r}_2*\pi_C/(\hat{r}_1 + \hat{r}_2)) + 1) - (-\hat{u}_1*beta + 2)*(K*\hat{u}_1/(\hat{u}_1 + \hat{u}_2) + \hat{r}_1*\pi_C/(\hat{r}_1 + \hat{r}_2))

## Find $J^*$

In [58]:
du1, du2, dr1, dr2 = sp.symbols('\Delta_{u_1}, \Delta_{u_2}, \Delta_{r_1}, \Delta_{r_2}')

In [59]:
dps1 = (du1*u2 - du2*u1)/(u1+u2)**2
dps2 = -dps1
dR1 = (dr1*r2 - dr2*r1)/(r1+r2)**2
dR2 = - dR1
L1 = K*psi1 + pc*R1
dL1 = K*dps1 + pc*dR1
L2 = K*psi2 + pc*R2
dL2 = - dL1

In [71]:
L1, L2 = sp.symbols('L_1, L_2')
dW =r1*dL1  + dr1*L1 + dr2*L2+ r2*dL2

In [78]:
expand(dW).collect([du1,du2,dr1,dr2])

K*\Delta_{u_1}*\hat{r}_1*\hat{u}_2/(\hat{u}_1**2 + 2*\hat{u}_1*\hat{u}_2 + \hat{u}_2**2) - K*\Delta_{u_1}*\hat{r}_2*\hat{u}_2/(\hat{u}_1**2 + 2*\hat{u}_1*\hat{u}_2 + \hat{u}_2**2) + \Delta_{r_1}*(L_1 + \hat{r}_1*\hat{r}_2*\pi_C/(\hat{r}_1**2 + 2*\hat{r}_1*\hat{r}_2 + \hat{r}_2**2) - \hat{r}_2**2*\pi_C/(\hat{r}_1**2 + 2*\hat{r}_1*\hat{r}_2 + \hat{r}_2**2)) + \Delta_{r_2}*(L_2 - \hat{r}_1**2*\pi_C/(\hat{r}_1**2 + 2*\hat{r}_1*\hat{r}_2 + \hat{r}_2**2) + \hat{r}_1*\hat{r}_2*\pi_C/(\hat{r}_1**2 + 2*\hat{r}_1*\hat{r}_2 + \hat{r}_2**2)) + \Delta_{u_2}*(-K*\hat{r}_1*\hat{u}_1/(\hat{u}_1**2 + 2*\hat{u}_1*\hat{u}_2 + \hat{u}_2**2) + K*\hat{r}_2*\hat{u}_1/(\hat{u}_1**2 + 2*\hat{u}_1*\hat{u}_2 + \hat{u}_2**2))

In [67]:
du1 = (1/W)*(L1*dr1 + dL1*(1+r1)) - (dW/(W**2))*(W*u1)

In [69]:
expand(du1).collect([du1,du2,dr1,dr2])

-K*\Delta_{u_1}*\hat{r}_1*\hat{u}_1*\hat{u}_2/(\hat{W}*\hat{u}_1**2 + 2*\hat{W}*\hat{u}_1*\hat{u}_2 + \hat{W}*\hat{u}_2**2) + K*\Delta_{u_1}*\hat{r}_1*\hat{u}_2/(\hat{W}*\hat{u}_1**2 + 2*\hat{W}*\hat{u}_1*\hat{u}_2 + \hat{W}*\hat{u}_2**2) + K*\Delta_{u_1}*\hat{r}_2*\hat{u}_1*\hat{u}_2/(\hat{W}*\hat{u}_1**2 + 2*\hat{W}*\hat{u}_1*\hat{u}_2 + \hat{W}*\hat{u}_2**2) + K*\Delta_{u_1}*\hat{u}_2/(\hat{W}*\hat{u}_1**2 + 2*\hat{W}*\hat{u}_1*\hat{u}_2 + \hat{W}*\hat{u}_2**2) + \Delta_{r_1}*(-K*\hat{u}_1**2/(\hat{W}*\hat{u}_1 + \hat{W}*\hat{u}_2) + K*\hat{u}_1/(\hat{W}*\hat{u}_1 + \hat{W}*\hat{u}_2) - \hat{r}_1*\hat{r}_2*\hat{u}_1*\pi_C/(\hat{W}*\hat{r}_1**2 + 2*\hat{W}*\hat{r}_1*\hat{r}_2 + \hat{W}*\hat{r}_2**2) + \hat{r}_1*\hat{r}_2*\pi_C/(\hat{W}*\hat{r}_1**2 + 2*\hat{W}*\hat{r}_1*\hat{r}_2 + \hat{W}*\hat{r}_2**2) - \hat{r}_1*\hat{u}_1*\pi_C/(\hat{W}*\hat{r}_1 + \hat{W}*\hat{r}_2) + \hat{r}_1*\pi_C/(\hat{W}*\hat{r}_1 + \hat{W}*\hat{r}_2) + \hat{r}_2**2*\hat{u}_1*\pi_C/(\hat{W}*\hat{r}_1**2 + 2*